# 명찰 시트 업무 자동화 코드

### 1. 패키지 설치

In [ ]:
!pip install python-pptx

In [ ]:
!pip install pandas

In [ ]:
!pip install numpy

### 2. 모듈 임포트

In [9]:
import pandas as pd
import numpy as np
from pptx import Presentation
from pptx.util import Inches
from pptx.dml.color import RGBColor
color_dict={'Red': '0xFF, 0x00, 0x00',
            'Green':'0x00, 0xFF, 0x00',
            'Blue':'0x00, 0x00, 0xFF',
            'Pink':'0xFF, 0x33, 0x99',
            'Orange':'0xFF, 0x7F, 0x00',
            'White': '0xFF, 0xFF, 0xFF'}

### 3. PPT 템플릿 로드

In [10]:
path_to_presentation = 'template.pptx'
prs = Presentation(path_to_presentation)

In [11]:
text_dict = dict()
for i, slide in enumerate(prs.slides):
    for k, shape in enumerate(slide.shapes):
        if not shape.has_text_frame:
            continue
        for p, paragraph in enumerate(shape.text_frame.paragraphs):
            for q, run in enumerate(paragraph.runs):
                text_dict[f"SLIDE_{i}_shape_{k}_paragraph_{p}_run_{q}"] = run.text

### 4. 참가자 명단 엑셀 데이터 로드

In [12]:
name_list = pd.read_csv("name_list.csv")
name_list.name = name_list.name.str.replace("", "  ")
name_list = name_list.fillna("")
empty_data = np.array([['BlankText'] * len(name_list.columns) for i in range(6 - len(name_list) % 6)])
empty_data[:, -1] = "White"
empty_df = pd.DataFrame(empty_data, columns=name_list.columns)
name_list = pd.concat((name_list, empty_df)).reset_index(drop=True)
name_list = name_list.replace("BlankText", "")
name_list

,position,name,subname,color
0,Grade 3,장 민 준,Kevin,Green
1,Grade 3,구 범 준,Eric,Green
2,Grade 4,서 지 우,Thor,Green
3,Grade 4,심 율,Mason,Green
4,Grade 2,강 인 서,Elsie,Pink
5,Grade 3,공 지 윤,Sunny,Pink
6,Grade 4,구 민 주,Irene,Pink
7,Grade 4,배 현 지,Rachel,Pink
8,Grade 4,신 지 은,Anny,Pink
9,Teacher,고 대 현,Daehyeon,Blue


### 5. 명단에 맞게 PPT 변환

In [13]:
position_index = 4
name_index = 5
subname_index = 6
color_index = 7

In [14]:
for i in range((len(name_list) // 6)):
    for j in range(6):
        prs.slides[i].shapes[position_index+8*j].text_frame.paragraphs[0].runs[0].text = name_list.position[i*6+j]
        prs.slides[i].shapes[name_index+8*j].text_frame.paragraphs[0].runs[0].text = name_list.name[i*6+j]
        prs.slides[i].shapes[subname_index+8*j].text_frame.paragraphs[0].runs[0].text = name_list.subname[i*6+j]
        prs.slides[i].shapes[color_index+8*j].fill.fore_color.rgb = RGBColor(*map(lambda v : int(v, 16),
                                                                                  color_dict[name_list.color[i*6+j]].split(',')))

In [15]:
prs.save("nametag.pptx")